# Welcome!
Today, we'll be using qBraid to understand how to embed classical images into quantum registers, the current embedding implementations, and the current limitations and problems at hand. 

## Getting Started on qBraid

### Step 1.
If you haven't already done so, please make a qBraid account and add the access code `EHNU6626` on the account.qbraid.com/account-details page. 


### Step 2.
Then click on the `Launch on qBraid` button in the README.md of this repository. The button will automatically clone the repository and take you to your *new* qBraid Lab integrated development environment. 

### Step 3.
Finally, install the qBraid-SDK environment via the qBraid Lab Environment Manager. On Lab you should see the `ENVS` icon on the right. The qBraid Lab Environment Manager is a robust package and virtual environment management system provided to qBraid end-users through a simple, intuitive graphical user interface. To expand the Environment Manager sidebar, click on Envs in the upper-right of the Lab console. My Environments are your currently installed environments. The qBraid Default environment and Microsoft Q# environment are installed by default.

Install environment
1. In the Environment Manager sidebar, click Add to view the environments available to install.

Choose the qBraid SDK, expand its panel, and click Install.

<img src="./_images/env_install.png">

3. Once the installation has started, the pannel is moved to the My Environments tab. Click Browse Environments to return to the My Environments tab and view its progress.



Browse Environments to return to the My Environments tab and view its progress.

<img src="./_images/env_installing.png">

4. When the installation is complete, the environment panel’s action button will switch from Installing… to Activate. Clicking Activate creates a new ipykernel, see Kernels for more.

<img src="./_images/kernel_activate.png">

To uninstall the environment, click on More, and then Uninstall. Learn more about qBraid Lab Environment Manager [here](https://qbraid-qbraid.readthedocs-hosted.com/en/stable/lab/environments.html#)

## Activate the qBraid SDK kernel
Under My Environments, choose the environment, and expand its pannel. Click Activate to activate the environment and create an associated ipykernel.

<img src="./_images/kernel_activate.png">

Switch notebook kernel
In the Launcher tab, under Notebooks, clicking on an ipykernel associated with an activated environment will automatically launch a Jupyter notebook (.ipynb file) using that kernel. In the upper-right of the newly created notebook, you can see which kernel is in use.

<img src="./_images/kernel_nb.png">

Clicking on the name of the current kernel, as circled above, will open the kernel selector, and allow you switch to any other active kernel.

<img src="./_images/kernel_switch.png">


Next we'll install tensorflow:

In [2]:
%pip install tensorflow
%pip install tensorflow-quantum


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 42.9 MB/s eta 0:00:0000:0100:01
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)
  Using cached google_api_core-1.21.0-py2.py3-none-any.whl (90 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 1.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.6 MB/s eta 0:00:00:00:01
  Using cached cachetools-4.2.4-py3-none-any.whl (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.8/641.8 kB 9.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 7.6 MB/s eta 0:00:0000:01
     

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
import qbraid
%matplotlib inline
# CONSTANTS
NUM_EXAMPLES=500
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
def plot_image(image):
  """Render image in jupyter notebook from numpy array."""
  plt.figure(figsize = (50, 15))
  plt.imshow(image)
  plt.show()

ModuleNotFoundError: No module named 'tensorflow_quantum'

# MNIST
The MNIST (Modified National Institute of Standards and Technology) database contains 70,000 28 x 28 images of handwritten digits from 0-9 and is seminal to machine learning. The MNIST handwritten dataset is the “Hello World” implementation for machine learning, and the dataset is used as a worldwide machine learning benchmark. 


## Starting off with a classical implementation
We will first load the data and apply a classical CNN (Convolutional Neural Network) to understand the mechanics of image classification.

### Loading the data
We load the data from tensorflow, a machine learning package developed by Google.

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale the images from [0,255] to the [0.0,1.0] range.
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

11490434/11490434 [==============================] - 0s 0us/step
Number of original training examples: 60000
Number of original test examples: 10000


In [7]:
y_train_onehot = tf.one_hot(y_train,10)
y_test_onehot = tf.one_hot(y_test,10)

In [14]:
def create_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, [3, 3], activation='relu', input_shape=(28,28,10)))
    model.add(tf.keras.layers.Conv2D(64, [3, 3], activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return model


model = create_classical_model()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.002),
               metrics=[tf.keras.metrics.CategoricalAccuracy()])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        2912      
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 64)        0         
                                                                 
 flatten_3 (Flatten)         (None, 9216)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               1179776   
                                                      

In [15]:
model.fit(x_train,
          y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test[:1000], y_test_onehot[:1000]))

cnn_results = model.evaluate(x_test, y_test_onehot)

Epoch 1/10


2022-11-25 07:07:02.142609: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


ValueError: in user code:

    File "/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/lib/python3.9/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        rais

### Analysis of classical layers using Tensorboard

## Quantum Computing and it's supposed benefits (Why quantum)

### Current challenge: embedding the circuit

### Methods of embedding the image into a circuit

### Other techniques (kernel methods etc).

### Some preliminary implementations


#### Analysis of current implementations using Tensorboard and qBraid SDK